In [11]:
# UDP to Avalon-MM bridge送受信

from socket import socket, AF_INET, SOCK_DGRAM
from struct import *

SOCKET_BUFFSIZE = 2048
ETHIO_SERVER = ("192.168.1.203", 16241)

def print_bin(label, bin):
	print(label, end=" : ")
	for b in bin: print(format(b, "02x"), end=" ")
	print("")


# Avalon-MMのライトパケットとリードパケットの作成

ETHIO_PACKET_FOURCC = b"AVMM"
ETHIO_PACKET_ENDCMD = b"\x7f\x00\xff\xff"

def data_padding(data):
	return data + b"\x00" * ((4 - len(data)) & 3)

def avm_memwr(sock, addr, wrbyte):
	send_data = ETHIO_PACKET_FOURCC
	send_data += pack("!BxHL", 0x44, len(wrbyte), addr) + data_padding(wrbyte)
	send_data += ETHIO_PACKET_ENDCMD
	print_bin("-> MEMWR", send_data)
	sock.sendto(send_data, ETHIO_SERVER)
	res, addr = sock.recvfrom(SOCKET_BUFFSIZE)
	print_bin("<-   RES", res)
	resp, size = unpack("!BxH", res[4:8])
	return size

def avm_memrd(sock, addr, readnum):
	send_data = ETHIO_PACKET_FOURCC
	send_data += pack("!BxHL", 0x54, readnum, addr)
	send_data += ETHIO_PACKET_ENDCMD
	print_bin("-> MEMWR", send_data)
	sock.sendto(send_data, ETHIO_SERVER)
	res, addr = sock.recvfrom(SOCKET_BUFFSIZE)
	print_bin("<-   RES", res)
	resp, size = unpack("!BxH", res[4:8])
	return res[8:8+size]


In [12]:
# Platform Designerモジュールのアクセス 

from datetime import *

with socket(AF_INET, SOCK_DGRAM) as sock:

	# SystemIDペリフェラルのリード

	rdbytes = avm_memrd(sock, 0x10000000, 8)
	sysid, timestamp = unpack('LL', rdbytes[0:8])
	dt = datetime.fromtimestamp(timestamp)
	print("System ID =", hex(sysid), ", Build", dt)

	# PIOペリフェラルのライト（LED点灯）

	avm_memwr(sock, 0x10000020, b"\x01\x00\x00\x00")
	print("UserLED[0] on")


-> MEMWR : 41 56 4d 4d 54 00 00 08 10 00 00 00 7f 00 ff ff 
<-   RES : 41 56 4d 4d d4 00 00 08 02 00 a0 72 6d b3 87 63 ff 00 00 00 
System ID = 0x72a00002 , Build 2022-12-01 04:47:57
-> MEMWR : 41 56 4d 4d 44 00 00 04 10 00 00 20 01 00 00 00 7f 00 ff ff 
<-   RES : 41 56 4d 4d c4 00 00 04 ff 00 00 00 
UserLED[0] on
